In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from modules.objects import DataframeColumnSelector
import statsmodels.api as sm
from sklearn.compose import make_column_transformer,ColumnTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn import set_config

df_train = pd.read_csv('raw_data/bb_train.csv')
df_dimension = pd.read_csv('raw_data/park_dimensions.csv')

In [ ]:
features_to_use =  [
                'is_batter_lefty',
                'is_pitcher_lefty',
                'bb_type',
                'bearing',
                'pitch_name',
                'inning',
                'outs_when_up',
                'balls',
                'strikes',
                'plate_x',
                'plate_z',
                'pitch_mph',
                'launch_speed',
                'launch_angle']

continuous_features = ['launch_angle',
                       'pitch_mph',
                       'plate_x',
                       'plate_z',
                       'launch_speed']

categorical_features = [i for i in features_to_use if i not in continuous_features]

categorical_transformers = []
continuous_transformers = []

for categorical_feature in categorical_features:

    ohe = OneHotEncoder(drop = 'first')

    categorical_feature_pipeline = Pipeline(
        steps = [
            ('one_hot_encoder',ohe)
                ]
                                            )       
    categorical_transformers.append((
            f"{categorical_feature}_pipeline",
            categorical_feature_pipeline,
            [categorical_feature]))   

for continuous_feature in continuous_features:

    if continuous_feature not in ['launch_angle','launch_speed']:

        scaler = StandardScaler()

        numeric_feature_pipeline = Pipeline(
            steps = [
                ('scaler',scaler)
            ]
        )

        continuous_transformers.append((
            f"{continuous_feature}_pipeline",
            numeric_feature_pipeline,
            [continuous_feature]))
    
    else:
    
        scaler = StandardScaler()
        imputer = IterativeImputer()

        numeric_feature_pipeline = Pipeline(
            steps = [
                ('scaler',scaler),
                ('imputer',imputer)])

        continuous_transformers.append((
            f"{continuous_feature}_pipeline",
            numeric_feature_pipeline,
            [continuous_feature]))

all_transformers = categorical_transformers + continuous_transformers
data_preprocessor = ColumnTransformer(all_transformers,remainder='passthrough')

transformation_pipeline = Pipeline(
    steps = [
        ('column_selector',DataframeColumnSelector(col_list=features_to_use)),
        ('preprocessor',data_preprocessor)
            ])
set_config(display="diagram")

transformation_pipeline

In [ ]:
to_logreg_data = transformation_pipeline.fit_transform(df_train)
pd.DataFrame(to_logreg_data.todense())

In [ ]:
transformation_pipeline.named_steps['preprocessor'].transformers_[3][1].get_feature_names_out()

In [ ]:
for i in range(len(transformation_pipeline.named_steps['preprocessor'].transformers_)):

    print([j for j in transformation_pipeline.named_steps['preprocessor'].transformers_[i][1].get_feature_names_out()])